[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/show4dstem/show4dstem_simple.ipynb)

# Show4DSTEM — Quick Demo

Synthetic 4D-STEM dataset with a bright-field disk, six first-order Bragg reflections,
six second-order spots, and scan-position-dependent intensity variation (mimicking
thickness/orientation changes across the sample). Poisson shot noise is applied for realism.

In [ ]:
# Install in Google Colab
try:
    import google.colab
    !pip install -q -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ quantem-widget
except ImportError:
    pass  # Not in Colab, skip

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %env ANYWIDGET_HMR=1
except Exception:
    pass  # autoreload unavailable (Colab Python 3.12+)

In [ ]:
import numpy as np


def make_4dstem(scan_x=16, scan_y=16, det_x=64, det_y=64):
    """4D-STEM dataset with BF disk, Bragg spots, and Kikuchi-like background."""
    data = np.zeros((scan_x, scan_y, det_x, det_y), dtype=np.float32)
    cy, cx = det_x // 2, det_y // 2
    yy, xx = np.mgrid[:det_x, :det_y]
    dist = np.sqrt((xx - cx)**2 + (yy - cy)**2)

    # Amorphous background (radial falloff)
    bg = 0.05 * np.exp(-dist / 30)

    for i in range(scan_x):
        for j in range(scan_y):
            dp = bg.copy()
            # BF disk with slight shift depending on scan position (beam tilt)
            shift_x = 0.3 * np.sin(2*np.pi*i/scan_x)
            shift_y = 0.3 * np.cos(2*np.pi*j/scan_y)
            bf_dist = np.sqrt((xx - cx - shift_x)**2 + (yy - cy - shift_y)**2)
            dp += np.where(bf_dist < 8, 1.0 + 0.2*np.cos(bf_dist*0.5), 0.0)

            # 6 first-order Bragg spots
            for k in range(6):
                angle = k * np.pi / 3
                sx = cx + 20 * np.cos(angle) + shift_x * 2
                sy = cy + 20 * np.sin(angle) + shift_y * 2
                # Intensity varies with scan position (thickness/orientation)
                intensity = 0.4 * (1 + 0.5*np.sin(2*np.pi*(i*np.cos(angle) + j*np.sin(angle))/scan_x))
                dp += intensity * np.exp(-((xx-sx)**2 + (yy-sy)**2) / (2*2.5**2))

            # Second-order spots (weaker)
            for k in range(6):
                angle = k * np.pi / 3 + np.pi / 6
                sx = cx + 35 * np.cos(angle)
                sy = cy + 35 * np.sin(angle)
                dp += 0.1 * np.exp(-((xx-sx)**2 + (yy-sy)**2) / (2*2**2))

            # Shot noise
            dp = np.maximum(dp, 0)
            dp = np.random.poisson(np.clip(dp * 200, 0, 1e6)).astype(np.float32) / 200
            data[i, j] = dp
    return data


data = make_4dstem()
print(f"Shape: {data.shape}, dtype: {data.dtype}")
print(f"Range: [{data.min():.3f}, {data.max():.3f}]")

Shape: (16, 16, 64, 64), dtype: float32
Range: [0.000, 1.530]


In [ ]:
from quantem.widget import Show4DSTEM

w = Show4DSTEM(data)
w.auto_detect_center()
w.roi_circle()
print(f"Detected center: ({w.center_row:.1f}, {w.center_col:.1f}), BF radius: {w.bf_radius:.1f}")
w

Detected center: (32.2, 31.8), BF radius: 9.9


Show4DSTEM(shape=(16, 16, 64, 64), sampling=(1.0 Å, 1.0 px), pos=(8, 8))

## Inspect Widget State

In [ ]:
w.summary()

Show4DSTEM
════════════════════════════════
Scan:     16×16 (1.00 Å/px)
Detector: 64×64 (1.0000 px/px)
Position: (8, 8)
Center:   (32.2, 31.8)  BF r=9.9 px
Display:  DC masked
ROI:      circle at (32.2, 31.8) r=5.0


## Real Data — Arina Detector

Load a 4D-STEM dataset from an Arina detector. The raw data is 256×256×192×192 (4.8 GB, over the 2B element GPU limit), so we bin k-space 2× to get 256×256×96×96 which fits on MPS GPU.

In [ ]:
import quantem

dataset = quantem.io.read_4dstem(
    '/Users/macbook/data/20251216_Korea_Sample_C1/KoreanSampleC1Bob_04_master.h5',
    file_type='arina'
)
print(f"Original: {dataset.array.shape}, {dataset.array.nbytes/1e9:.1f} GB")

In [ ]:
from quantem.widget import Show4DSTEM

widget = Show4DSTEM(dataset)
widget